# Глобальные градиентные методы оптимизации

#### Подключим необходимые библиотеки

In [1]:
import numpy as np
from interval import imath
from interval import fpu
from interval import interval
from functions import Functions

#### Вспомогательные функции, которые пригодятся далее

In [2]:
# возвращает левую границу интервала
def left(interval):
    return fpu.max(interval)[0]

# возвращает правую границу интервала
def right(interval):
    return fpu.max(interval)[1]

# возвращает середину интервала
def mid(interval):
    return (fpu.max(interval)[0] + fpu.max(interval)[1]) / 2

# возвращает евклидово расстояние между 2 векторами
def dist(p_1, p_2):
    return np.sqrt(np.sum(np.square(np.array(p_1) - np.array(p_2))))

#### Напишем алгоритм, который будет находить локальный минимум функции вдоль заданного направления.
#### Алгоритм будет основываться на методе "золотого сечения", так как является наиболее эффективным среди основных методов одномерной оптимизации.

In [3]:
def GoldenRatio(func_index, index, a, b, p, e_d, e_f):  # f(function), i(index of direction), a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    F = Functions[func_index * 2]
    phi = (1 + np.sqrt(5)) / 2  # constant of golden ratio
    x_1 = b - (b - a) / phi
    x_2 = a + (b - a) / phi
    p_1 = p.copy()  # current point
    p_2 = p.copy()  # current point
    p_1[index] = x_1
    p_2[index] = x_2
    f_1 = F(p_1)  # value in 1-st point
    f_2 = F(p_2)  # value in 2-nd point
    while (b - a > e_d) | (abs(f_1 - f_2) > e_f):  # termination criteria
        if f_1 <= f_2:
            b = x_2
            x_2 = x_1
            x_1 = b - (b - a) / phi
            p_1[index] = x_1
            p_2[index] = x_2
            f_2 = f_1
            f_1 = F(p_1)
        else:
            a = x_1
            x_1 = x_2
            x_2 = a + (b - a) / phi
            p_1[index] = x_1
            p_2[index] = x_2
            f_1 = f_2
            f_2 = F(p_2)

    best_point = []
    for i in range(len(p)):
        best_point.append((p_1[i] + p_2[i]) / 2)

    if F(best_point) > F(p):
        best_point = p
        
    return best_point  # point of extremum with error e_d

#### Теперь реализуем алгоритм Moore-Skelboe, который находит глобальный минимум функции вдоль направления коллинеарного главным осям.

In [4]:
def MooreSkelboe(func_index, index, a, b, p, e_d,
                 e_f):  # func_index(number of function in list of functions), index(index of direction),
    # a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    F = Functions[func_index * 2 + 1]
    interval_d = []
    for i in range(len(p)):
        interval_d.append(interval[p[i], p[i]])
    interval_d[index] = interval[a, b]

    interval_f = F(interval_d)
    set_of_intervals = [[interval_d, interval_f]]
    U = right(interval_f)
    w_f = right(interval_f) - left(interval_f)
    w_d = right(interval_d[index]) - left(interval_d[index])
    best_interval = set_of_intervals[0]
    while (w_d > e_d) | (w_f > e_f):
        set_of_intervals.pop(0)
        mid_p = mid(best_interval[0][index])
        interval_1 = best_interval[0].copy()
        interval_2 = best_interval[0].copy()
        interval_1[index] = interval[left(best_interval[0][index]), mid_p]
        interval_1_f = F(interval_1)
        interval_2[index] = interval[mid_p, right(best_interval[0][index])]
        interval_2_f = F(interval_2)
        U = min(U, right(interval_1_f))
        U = min(U, right(interval_2_f))


        if (len(set_of_intervals)>0) and (U<left(set_of_intervals[-1][1])):
            l = 0
            r = len(set_of_intervals) - 1
            while l < r:
                m = int((l + r) / 2)
                if left(set_of_intervals[m][1]) > U:
                   r = m
                else:
                    l = m + 1
            set_of_intervals = set_of_intervals[:l]

        set_of_intervals.append([interval_1, interval_1_f])
        set_of_intervals.append([interval_2, interval_2_f])
        set_of_intervals.sort(key=lambda item: left(item[1]))


        best_interval = set_of_intervals[0]
        w_f = right(best_interval[1]) - left(best_interval[1])
        w_d = right(best_interval[0][index]) - left(best_interval[0][index])

    best_point = []
    for i in range(len(p)):
        best_point.append(mid(best_interval[0][i]))

    return best_point

#### Функция которая реализует метод координатного спуска: последовательно ищет минимум вдоль направлений коллинеарным главным осям и останавливается, когда улучшение точности становится малым.

In [5]:
def CoordinateDescent(func_index, D, p, e_d, e_f, method):  # F(function), D(set), p(start point), e(error)
    while True:
        p_0 = p
        for index in range(len(p)):
            p = method(func_index, index, D[index][0], D[index][1], p, e_d, e_f)
        if (dist(p_0, p) < e_d) & (Functions[func_index * 2](p_0) - Functions[func_index * 2](p) < e_f):
            break

    best_point = p
    best_value = Functions[func_index * 2](p)

    return best_point, best_value

#### Функция для вывода результата работы алгоритма

In [6]:
def CoordinateResult(func_num, D, p, e_d, e_f):
    p_1, v_1 = CoordinateDescent(func_num, D, p, e_d, e_f, GoldenRatio)  # point of minimum
    p_2, v_2 = CoordinateDescent(func_num, D, p, e_d, e_f, MooreSkelboe)  # point of minimum
    print("Результат тестирования:")
    print("Golden Ratio:", v_1)
    print("Moore-Skelboe:", v_2)


In [7]:
def print_result1(func_num, D, p, e_d, e_f):
    p_1, v_1 = CoordinateDescent(func_num, D, p, e_d, e_f, GoldenRatio)  # point of minimum
    print("Результат тестирования:")
    print("Golden Ratio:", v_1)

In [8]:
def print_result2(func_num, D, p, e_d, e_f):
    p_2, v_2 = CoordinateDescent(func_num, D, p, e_d, e_f, MooreSkelboe)  # point of minimum
    print("Результат тестирования:")
    print("Moore-Skelboe:", v_2)

#### Продемонстрируем работу методов на известных тестовых функциях глобальной оптимизации, применяя их в координатном спуске

In [11]:
# Функция Растригина
n = 100
l = -10
r = 10
D = [[l, r]] * n
p = [1] * n
# CoordinateResult(5, D, p, 0.001, 0.001)
print_result1(5, D, p, 0.001, 0.001)

Результат тестирования:
Golden Ratio: 99.49642551412171


In [12]:
# Функция Экли
n = 100
l = -10
r = 10
D = [[l, r]] * n
p = [2] * n
# CoordinateResult(15, D, p, 0.001, 0.001)

print_result1(15, D, p, 0.001, 0.001)
# print_result2(15, D, p, 0.001, 0.001)

Результат тестирования:
Golden Ratio: 3.5744545080266503


### Реализуем теперь градиентные алгоритмы

In [9]:
def Gradient(D, p, f, step):  # градиент с отражением
    gradient = np.array([0.] * len(p))
    p_ = p.copy()
    for i in range(len(p)):
        p_[i] += step
        gradient[i] = (f(p_) - f(p)) / step

        if (abs(p[i] - D[i][0]) < step) & (gradient[i] < 0):
            gradient[i] = 0
        if (abs(p[i] - D[i][1]) < step) & (gradient[i] > 0):
            gradient[i] = 0
        p_[i] -= step
    return gradient

In [10]:
def GradientDescent(func_index, D, p, e_d, e_f,
                method):  # F(function), D(set), p(start point), e(error)
    F = Functions[func_index * 2]
    gradient = Gradient(D, p, F, e_d)
    while np.linalg.norm(gradient) > e_f:
        p_0 = p
        p = method(func_index, gradient, D, p, e_d, e_f)
        gradient = Gradient(D, p, F, e_d)
        if dist(p, p_0) < e_d or abs(F(p) - F(p_0))<e_f:
            break

    best_point = p
    best_value = Functions[func_index * 2](p)
    return best_point, best_value

In [11]:
def Borders(p, gradient, D):
    max_t = np.inf
    min_t = -np.inf
    dir = gradient / np.linalg.norm(gradient)
    for i in range(len(p)):
        if dir[i] > 0:
            max_t = min((D[i][1] - p[i]) / dir[i], max_t)
            min_t = max((D[i][0] - p[i]) / dir[i], min_t)
        elif dir[i] < 0:
            min_t = max((D[i][1] - p[i]) / dir[i], min_t)
            max_t = min((D[i][0] - p[i]) / dir[i], max_t)

    borders = [p + dir * min_t, p + dir * max_t]
    return borders

In [12]:
def GradientGoldenRatio(func_index, gradient, D, p, e_d,
                 e_f):  # f(function), i(index of direction),
    # a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    F = Functions[func_index * 2]
    phi = (1 + np.sqrt(5)) / 2  # constant of golden ratio
    a = Borders(p, gradient, D)[0]
    b = Borders(p, gradient, D)[1]
    x_1 = b - (b - a) / phi
    x_2 = a + (b - a) / phi
    p_1 = x_1
    p_2 = x_2
    f_1 = F(p_1)  # value in 1-st point
    f_2 = F(p_2)  # value in 2-nd point
    while (dist(b, a) > e_d) | (abs(f_1 - f_2) > e_f):  # termination criteria
        if f_1 <= f_2:
            b = x_2
            x_2 = x_1
            x_1 = b - (b - a) / phi

            p_1 = x_1
            p_2 = x_2

            f_2 = f_1
            f_1 = F(p_1)
        else:
            a = x_1
            x_1 = x_2
            x_2 = a + (b - a) / phi

            p_1 = x_1
            p_2 = x_2

            f_1 = f_2
            f_2 = F(p_2)

    best_point = []
    for i in range(len(p)):
        best_point.append((p_1[i] + p_2[i]) / 2)

    return best_point  # point of extremum with error e_d

In [13]:
def GradientMooreSkelboe(func_index, gradient, D, p, e_d,
                  e_f):  # p(current point), e_d(error of d), e_f(error of f)
    F = Functions[func_index * 2 + 1]
    interval_d = [None] * len(p)

    a = Borders(p, gradient, D)[0]
    b = Borders(p, gradient, D)[1]

    for i in range(len(p)):
        interval_d[i] = interval[a[i], b[i]]

    interval_f = F(interval_d)
    set_of_intervals = [[interval_d, interval_f]]
    U = right(interval_f)
    w_f = right(interval_f) - left(interval_f)
    for index in range(len(p)):
        w_d = np.square(right(interval_d[index]) - left(interval_d[index]))
    w_d = np.sqrt(w_d)
    best_interval = set_of_intervals[0]
    while (w_d > e_d) | (w_f > e_f):
        set_of_intervals.pop(0)
        mid_p = [None] * len(p)
        for index in range(len(p)):
            mid_p[index] = mid(best_interval[0][index])

        interval_1 = best_interval[0].copy()
        interval_2 = best_interval[0].copy()
        for index in range(len(p)):
            interval_1[index] = interval[left(best_interval[0][index]), mid_p[index]]
            interval_2[index] = interval[mid_p[index], right(best_interval[0][index])]

        interval_1_f = F(interval_1)
        interval_2_f = F(interval_2)

        U = min(U, right(interval_1_f))
        U = min(U, right(interval_2_f))

        if (len(set_of_intervals)>0) and (U<left(set_of_intervals[-1][1])):
            l = 0
            r = len(set_of_intervals) - 1
            while l < r:
                m = int((l + r) / 2)
                if left(set_of_intervals[m][1]) > U:
                   r = m
                else:
                    l = m + 1
            set_of_intervals = set_of_intervals[:l]


        set_of_intervals.append([interval_1, interval_1_f])
        set_of_intervals.append([interval_2, interval_2_f])

        set_of_intervals.sort(key=lambda item: left(item[1]))
        best_interval = set_of_intervals[0]
        w_f = right(best_interval[1]) - left(best_interval[1])

        for index in range(len(p)):
            w_d = np.square(right(best_interval[0][index]) - left(best_interval[0][index]))
        w_d = np.sqrt(w_d)
        # print(w_f, w_d)
    

    best_point = []
    for i in range(len(p)):
        best_point.append(mid(best_interval[0][i]))

    return best_point

#### А теперь продемонстрируем работу методов на известных тестовых функциях глобальной оптимизации, применяя их в градиентном спуске

In [14]:
def GradientResult(func_num, D, p, e_d, e_f):
    p_1, v_1 = GradientDescent(func_num, D, p, e_d, e_f, GradientGoldenRatio)  # point of minimum
    p_2, v_2 = GradientDescent(func_num, D, p, e_d, e_f, GradientMooreSkelboe)  # point of minimum
    print("Результат тестирования:")
    print("Golden Ratio:", v_1)
    print("Moore-Skelboe:", v_2)


In [24]:
# Функция Растригина
n = 10
l = -10
r = 10
D = [[l, r]] * n
p = [1] * n
# GradientResult(5, D, p, 0.001, 0.001)

# p_1, v_1 = GradientDescent(5, D, p, 0.001, 0.001, GradientGoldenRatio)  # point of minimum
# print(v_1)

# p_1, v_1 = GradientDescent(5, D, p, 0.001, 0.001, GradientMooreSkelboe)  # point of minimum
# print(v_1)

9.949606847337455


In [26]:
# Функция Экли
n = 10
l = -10
r = 10
D = [[l, r]] * n
p = [2] * n
# GradientResult(15, D, p, 0.001, 0.001)

p_1, v_1 = GradientDescent(15, D, p, 0.001, 0.001, GradientGoldenRatio)  # point of minimum
print(v_1)

# p_1, v_1 = GradientDescent(15, D, p, 0.001, 0.001, GradientMooreSkelboe)  # point of minimum
# print(v_1)

0.00023862367055516032


In [15]:
def Multidimensional_Moore_Skelboe(func_index, D, e_d, e_f):
    F = Functions[func_index * 2 + 1]
    interval_f = F(D)
    list_of_intervals = [[D, interval_f]]

    U = right(interval_f)
    w_f = right(interval_f) - left(interval_f)
    best_set = list_of_intervals[0]

    w_d = right(best_set[0][0]) - left(best_set[0][0])
    for i in range(len(best_set[0])):
        if w_d < right(best_set[0][i]) - left(best_set[0][i]):
            w_d = right(best_set[0][i]) - left(best_set[0][i])

    while (w_d > e_d) | (w_f > e_f):
        list_of_intervals.pop(0)
        set_1 = best_set[0].copy()
        set_2 = best_set[0].copy()
        sep_index = 0
        w_d = right(best_set[0][0]) - left(best_set[0][0])
        for i in range(len(best_set[0])):
            if w_d < right(best_set[0][i]) - left(best_set[0][i]):
                w_d = right(best_set[0][i]) - left(best_set[0][i])
                sep_index = i

        mid = (left(best_set[0][sep_index]) + right(
            best_set[0][sep_index])) / 2

        set_1[sep_index] = interval[left(set_1[sep_index]), mid]
        set_2[sep_index] = interval[mid, right(set_2[sep_index])]

        interval_1f = F(set_1)
        interval_2f = F(set_2)
        U = min(U, right(interval_1f))
        U = min(U, right(interval_2f))

        if (len(list_of_intervals)>0) and (U<left(list_of_intervals[-1][1])):
            l = 0
            r = len(list_of_intervals) - 1
            while l < r:
                m = int((l + r) / 2)
                if left(list_of_intervals[m][1]) > U:
                   r = m
                else:
                    l = m + 1
            list_of_intervals = list_of_intervals[:l]

        list_of_intervals.append([set_1, interval_1f])
        list_of_intervals.append([set_2, interval_2f])
        list_of_intervals.sort(key=lambda item: left(item[1]))
        
        best_set = list_of_intervals[0]
        w_f = right(best_set[1]) - left(best_set[1])

    min_value = (right(best_set[1]) + left(best_set[1])) / 2
    return min_value

In [19]:
n = 1
l = -10
r = 10
D = [interval[l, r]] * n

Multidimensional_Moore_Skelboe(23, D, 0.001, 0.001)

3.375967821545034e-05